![Hand with calculator](calculator.jpg)

Did you know that the average return from investing in stocks is [10% per year](https://www.nerdwallet.com/article/investing/average-stock-market-return) (not accounting for inflation)? But who wants to be average?! 

You have been asked to support an investment firm by analyzing trends in high-growth companies. They are interested in understanding which industries are producing the highest valuations and the rate at which new high-value companies are emerging. Providing them with this information gives them a competitive insight as to industry trends and how they should structure their portfolio looking forward.

You have been given access to their `unicorns` database, which contains the following tables:

## dates
| Column       | Description                                  |
|------------- |--------------------------------------------- |
| `company_id`   | A unique ID for the company.                 |
| `date_joined` | The date that the company became a unicorn.  |
| `year_founded` | The year that the company was founded.       |

## funding
| Column           | Description                                  |
|----------------- |--------------------------------------------- |
| `company_id`       | A unique ID for the company.                 |
| `valuation`        | Company value in US dollars.                 |
| `funding`          | The amount of funding raised in US dollars.  |
| `select_investors` | A list of key investors in the company.      |

## industries
| Column       | Description                                  |
|------------- |--------------------------------------------- |
| `company_id`   | A unique ID for the company.                 |
| `industry`     | The industry that the company operates in.   |

## companies
| Column       | Description                                       |
|------------- |-------------------------------------------------- |
| `company_id`   | A unique ID for the company.                      |
| `company`      | The name of the company.                          |
| `city`         | The city where the company is headquartered.      |
| `country`      | The country where the company is headquartered.   |
| `continent`    | The continent where the company is headquartered. |


# The output

Your query should return a table in the following format:
| industry  | year | num\_unicorns       | average\_valuation\_billions |
| --------- | ---- | ------------------- | ---------------------------- |
| industry1 | 2021 |        ---          |             ---              |
| industry2 | 2020 |        ---          |             ---              |
| industry3 | 2019 |        ---          |             ---              |
| industry1 | 2021 |        ---          |             ---              |
| industry2 | 2020 |        ---          |             ---              |
| industry3 | 2019 |        ---          |             ---              |
| industry1 | 2021 |        ---          |             ---              |
| industry2 | 2020 |        ---          |             ---              |
| industry3 | 2019 |        ---          |             ---              |

Where `industry1`, `industry2`, and `industry3` are the three top-performing industries.

In [19]:
-- Identify the three best-performing industries based on the number of new unicorns created in 2019, 2020, and 2021 combined.
with top_by_volume as (
select 
	industry,
	count(I.company_id) Total_company
from 
	industries I join public.dates D on I.company_id = D.company_id
where
	extract (year from D.date_joined) in ('2019','2020','2021')
group by 
	industry
order by
	total_company desc
limit 3
	),


yearly_ranking_val as (
 select 
	extract( year from d.date_joined) as year,
	industry,
	count(I.company_id) num_unicorn,
	avg(F.valuation) as average_valuation
from 
	industries I join public.dates D on I.company_id = D.company_id
				 join  public.funding F on I.company_id = F.company_id
group by 
	industry, year
)

select
	year,
	industry,
	num_unicorn,
	ROUND(AVG(average_valuation / 1000000000), 2) AS average_valuation_billions
from 
	yearly_ranking_val
where
	year in ('2019','2020','2021')
	and  industry in (select industry
	from top_by_volume )
group by 
	industry,num_unicorn,year
order by average_valuation_billions desc
